In [ ]:
import os
import numpy as np, pandas as pd
import json


In [ ]:
pd.options.display.max_rows = 100
# unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
# users = pd.read_csv('ml-latest/users.csv', sep=',',
#  header=None, names=unames, engine='python', encoding='latin-1')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_table('ml-1m/ratings.dat', sep='::',
 header=None, names=rnames, engine='python', encoding='latin-1')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('ml-1m/movies.dat', sep='::',
 header=None, names=mnames, engine='python', encoding='latin-1')

In [ ]:
ratings = ratings[ratings["rating"] >= 4]  # Remove disliked movies
ratings = ratings.merge(movies, on="movie_id")  # Add in titles
merged_ratings = pd.merge(ratings[['user_id', 'movie_id', 'rating']], movies[['movie_id', 'title','genres']], on='movie_id')
merged_ratings = merged_ratings.drop_duplicates(subset=['user_id', 'title'])

In [ ]:
count =0 
instruction = "Given a list of liked movies, recommend 4 more movies the user would like."
prompt_list = []
for _, user in merged_ratings.groupby("user_id"): 
   
        if user.shape[0] >= 10:
                count +=1
                
                # Get amount of 10 movie blocks in user.shape[0]
                num_blocks = int(user.shape[0]/10)
                # For each block, create a fine-tuning example
                for i in range(num_blocks):
                        # Get 10 movies
                        movies = user.iloc[i*10:(i+1)*10]
                        # fitst 6 movies are the input, last 4 are the output in string form
                        input_str = movies.iloc[:6].title.str.cat(sep=', ')
                        output_str = movies.iloc[6:].title.str.cat(sep=', ')
                        
                        prompt = {
                                "instruction": f"{instruction}",
                                "input": f"{input_str} =>",
                                "output": f"{output_str}"
                                }
                        prompt_list.append(prompt)


                        
        
data_len = len(prompt_list)           
                
print('Prepared fine-tuning and evaluation datasets')     
print(f'Tuning set is of size {data_len}, given even amount for train/eval, the respective sizes of sets are {int(data_len*0.85)} for training and {int(data_len*0.15)} for testing')

In [ ]:
with open('llama_train.json', 'w') as outfile:
    json.dump(prompt_list[:int(data_len*0.85)], outfile)

with open('llama_eval.json', 'w') as outfile:
    json.dump(prompt_list[int(data_len*0.85):], outfile)